In [1]:
# Necessary libraries that will be employed
import itertools
from copy import deepcopy
from dataclasses import dataclass
from typing import List
import numpy as np

from sudoku_func import *

#!{sys.executable} -m pip install clingo
#!{sys.executable} -m pip install python-sat
import clingo
import pysat
from pysat.formula import CNF
from pysat.solvers import MinisatGH

from sudoku_func import pretty_repr

**Get positive diagonal elements**

In [2]:
def get_positive_diagonal_elements(matrix, j):
    """
    Return a list of the positive diagonal elements of the matrix.
    """
    N = len(matrix)
    idx = range(0, min(N, j) + 1)
    for t in idx:
        matrix[t][j - t] = 1
    
    return matrix

In [3]:
def get_negative_diagonal_elements(matrix, i):
    """
    Return a list of the negative diagonal elements of the matrix.
    """
    N = len(matrix)
    idx = range(0, min(N, N - i))
    for t in idx:
        matrix[i + t][t] = 1
    
    return matrix

In [4]:
matrix = [[0, 0, 0, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 0, 0, 0, 0],
]
# matrix = np.zeros((9, 9), dtype=int)
get_positive_diagonal_elements(matrix, 3)

[[0, 0, 0, 1, 0, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0]]

In [5]:
matrix = [[0, 0, 0, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 0, 0, 0, 0],
]
# matrix = np.zeros((9, 9), dtype=int)
get_negative_diagonal_elements(matrix, 7)

[[0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 0, 0, 0]]

In [6]:
def zigzag_southwest_to_northeast(matrix):
    n = len(matrix)
    
    for i in range(0, n):
        t = 0
        while t < n and t <= i:
            matrix[i - t][t] = i
            t += 1
    
    for j in range(1, n):
        t = 0
        while t<= (n - 1) and (j + t < n):
            matrix[n - 1 - t][j + t] = j
            t += 1

    return matrix

In [7]:
matrix = [[0, 0, 0, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 0, 0, 0, 0],
]
zigzag_southwest_to_northeast(matrix)

[[0, 1, 2, 3, 4, 5, 6, 7, 8],
 [1, 2, 3, 4, 5, 6, 7, 8, 1],
 [2, 3, 4, 5, 6, 7, 8, 1, 2],
 [3, 4, 5, 6, 7, 8, 1, 2, 3],
 [4, 5, 6, 7, 8, 1, 2, 3, 4],
 [5, 6, 7, 8, 1, 2, 3, 4, 5],
 [6, 7, 8, 1, 2, 3, 4, 5, 6],
 [7, 8, 1, 2, 3, 4, 5, 6, 7],
 [8, 1, 2, 3, 4, 5, 6, 7, 8]]

In [8]:
def zigzag_northwest_to_southeast(matrix):
    n = len(matrix)
    
    for i in range(0, n):
        t = 0
        while t<= (n - 1) and (i + t < n):
            matrix[i + t][t] = i
            t += 1

    for j in range(1, n):
    # for j in [3]:
        t = 0
        while t < n and t + j < n:
            matrix[t][j + t] = j
            t += 1
    
    return matrix

In [9]:
matrix = [[0, 0, 0, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 0, 0, 0, 0],
]
zigzag_northwest_to_southeast(matrix)

[[0, 1, 2, 3, 4, 5, 6, 7, 8],
 [1, 0, 1, 2, 3, 4, 5, 6, 7],
 [2, 1, 0, 1, 2, 3, 4, 5, 6],
 [3, 2, 1, 0, 1, 2, 3, 4, 5],
 [4, 3, 2, 1, 0, 1, 2, 3, 4],
 [5, 4, 3, 2, 1, 0, 1, 2, 3],
 [6, 5, 4, 3, 2, 1, 0, 1, 2],
 [7, 6, 5, 4, 3, 2, 1, 0, 1],
 [8, 7, 6, 5, 4, 3, 2, 1, 0]]

**Neighborhood of a cell**

In [10]:
import itertools

matrix = np.zeros((9, 9), dtype=int)
n = len(matrix)
matrix = matrix.astype(str)

i, j = 1, 1

somelists = [
   [max(i - 1, 0), i, min(n - 1, i + 1)],
   [max(j - 1, 0), j, min(n - 1, j + 1)],
]
for element in itertools.product(*somelists):
    matrix[element[0]][element[1]] = "N"

print(matrix)

[['N' 'N' 'N' '0' '0' '0' '0' '0' '0']
 ['N' 'N' 'N' '0' '0' '0' '0' '0' '0']
 ['N' 'N' 'N' '0' '0' '0' '0' '0' '0']
 ['0' '0' '0' '0' '0' '0' '0' '0' '0']
 ['0' '0' '0' '0' '0' '0' '0' '0' '0']
 ['0' '0' '0' '0' '0' '0' '0' '0' '0']
 ['0' '0' '0' '0' '0' '0' '0' '0' '0']
 ['0' '0' '0' '0' '0' '0' '0' '0' '0']
 ['0' '0' '0' '0' '0' '0' '0' '0' '0']]


**k-k blocks**

In [11]:
k = 4
matrix = np.zeros((k * k, k * k), dtype=object)
# matrix = matrix.astype(str)

for I in range(k):
    for J in range(k):
        i_range = range(I * k, (I + 1) * k)
        j_range = range(J * k, (J + 1) * k)
        for i in i_range:
            for j in j_range:
                matrix[i][j] = str(tuple((I, J)))

print(pretty_repr(matrix, k))

.-------------.-------------.-------------.-------------.
| (0, 0) (0, 0) (0, 0) (0, 0) | (0, 1) (0, 1) (0, 1) (0, 1) | (0, 2) (0, 2) (0, 2) (0, 2) | (0, 3) (0, 3) (0, 3) (0, 3) | 
| (0, 0) (0, 0) (0, 0) (0, 0) | (0, 1) (0, 1) (0, 1) (0, 1) | (0, 2) (0, 2) (0, 2) (0, 2) | (0, 3) (0, 3) (0, 3) (0, 3) | 
| (0, 0) (0, 0) (0, 0) (0, 0) | (0, 1) (0, 1) (0, 1) (0, 1) | (0, 2) (0, 2) (0, 2) (0, 2) | (0, 3) (0, 3) (0, 3) (0, 3) | 
| (0, 0) (0, 0) (0, 0) (0, 0) | (0, 1) (0, 1) (0, 1) (0, 1) | (0, 2) (0, 2) (0, 2) (0, 2) | (0, 3) (0, 3) (0, 3) (0, 3) | 
.-------------.-------------.-------------.-------------.
| (1, 0) (1, 0) (1, 0) (1, 0) | (1, 1) (1, 1) (1, 1) (1, 1) | (1, 2) (1, 2) (1, 2) (1, 2) | (1, 3) (1, 3) (1, 3) (1, 3) | 
| (1, 0) (1, 0) (1, 0) (1, 0) | (1, 1) (1, 1) (1, 1) (1, 1) | (1, 2) (1, 2) (1, 2) (1, 2) | (1, 3) (1, 3) (1, 3) (1, 3) | 
| (1, 0) (1, 0) (1, 0) (1, 0) | (1, 1) (1, 1) (1, 1) (1, 1) | (1, 2) (1, 2) (1, 2) (1, 2) | (1, 3) (1, 3) (1, 3) (1, 3) | 
| (1, 0) (1, 0) (1, 0) 

### Trying to solve a Sudoku

In [12]:
sudoku=[[9, 5, 6, 0, 0, 4, 0, 0, 0],
        [3, 7, 8, 9, 5, 6, 0, 0, 0],
        [0, 0, 4, 3, 7, 8, 0, 0, 0],
        [8, 9, 5, 6, 0, 0, 0, 0, 0],
        [4, 3, 7, 8, 9, 5, 0, 0, 0],
        [6, 0, 0, 4, 3, 0, 0, 0, 0],
        [7, 8, 9, 5, 6, 0, 0, 0, 0],
        [0, 4, 3, 7, 8, 0, 0, 0, 0],
        [5, 6, 0, 0, 4, 0, 0, 0, 0]]

In [32]:
sudoku=[[0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 3, 0, 0, 8, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 4, 0, 0, 0, 5],
        [0, 0, 0, 0, 0, 0, 0, 2, 7],
        [0, 0, 0, 0, 0, 0, 3, 0, 0],
        [0, 0, 0, 0, 0, 0, 9, 0, 0],
        [0, 0, 0, 0, 5, 6, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0]]

In [33]:
N = len(sudoku)
k = int(N ** 0.5)

In [34]:
N, k

(9, 3)

In [35]:
def describe_cnf(cnf):
    """Prints relevant information about the CNF."""
    print("Number of variables:", cnf.nv)
    print("Number of clauses:", len(cnf.clauses))
    print("Number of literals:", sum(len(clause) for clause in cnf.clauses))
    print("Number of clauses per variable:", cnf.nv / len(cnf.clauses))

In [46]:
def solve_sudoku_SAT(sudoku, k=3):
    N = len(sudoku)
    from pysat.formula import CNF, IDPool

    # Notation: P_ijl is a propositional variable that is true iff the cell at row i, column j has value l.
    vpool = IDPool(start_from=1)
    P = lambda i, j, l: vpool.id('P{0}@{1}@{2}'.format(i, j, l))

    # variables = {
    #     i * j * l: P(i, j, l) for i in range(N) for j in range(N) for l in range(1, N+1)
    #     }

    variables = {
        P(i, j, l): (i, j, l) for i in range(N) for j in range(N) for l in range(1, N+1)
    }
    
    # indices = [(i, j, k) for i in range(N) for j in range(N) for l in range(1, N+1)]
    # indices = [P(i, j, k) for i in range(N) for j in range(N) for l in range(1, N+1)]

    # initialize the formula
    cnf = CNF()

    # Constrain 1A: Each cell must contain a value between 1 and N.
    for i in range(N):
        for j in range(N):
            cnf.append([P(i, j, l) for l in range(1, N + 1)])

    # Constrain 1B; Each cell contains at most 1 number.
    for i in range(N):
        for j in range(N):
            for l in range(1, N):
                for m in range(l + 1, N + 1):
                    cnf.append([-P(i, j, l), -P(i, j, m)])

    # Constraint 2: Each row contains all numbers.
    # For each row i, for each number l, there must be a cell in row i that contains l.
    for i in range(N):
        for l in range(1, N + 1):
            cnf.append([P(i, j, l) for j in range(N)])

    # Constaint 3: Each column contains all numbers.
    # For each column j, for each number l, there must be a cell in column j that contains l.
    for j in range(N):
        for l in range(1, N + 1):
            cnf.append([P(i, j, l) for i in range(N)])

    # Constraint 4: Each kxk sub-grid contains all numbers.
    for I in range(k):
        for J in range(k):
            i_range = range(I * k, (I + 1) * k)
            j_range = range(J * k, (J + 1) * k)
            for l in range(1, N + 1):
                cnf.append([P(i, j, l) for i in i_range for j in j_range])

    # Constraint 5: For every cell, its neighborhood should not contain
    # the same element as the cell
    for i in range(N):
        for j in range(N):
            ij_ranges = [
                [max(i - 1, 0), i, min(n - 1, i + 1)],
                [max(j - 1, 0), j, min(n - 1, j + 1)],
            ]
            ngbd = itertools.product(*ij_ranges)
            for l in range(1, N + 1):
                for (i_, j_) in ngbd:
                    if i_ != i or j_ != j:
                        cnf.append([-P(i, j, l), -P(i_, j_, l)])

    # Constraint 6: Knight's constraint
    for i, j in itertools.product(range(0, k**2), repeat=2):
        for i_, j_ in [(1, 2), (1, -2), (-1, 2), (-1, -2),
                       (2, 1), (-2, 1), (2, -1), (-2, -1)]:
            if i_ != 0 or j_ != 0:
                i_ += i
                j_ += j
                
                for l in range(1, N + 1):
                    if 0 <= i_ < N and 0 <= j_ < N:
                        cnf.append([-P(i, j, l), -P(i_, j_, l)])
    
    # Contraint 7: Retain values provided in the input sudoku.
    for i in range(N):
        for j in range(N):
            if sudoku[i][j] != 0:
                cnf.append([P(i, j, sudoku[i][j])])

    # print characteristics of the formula
    print("CNF details:")
    describe_cnf(cnf)
    print()

    # solve the formula
    from pysat.solvers import MinisatGH
    solver = MinisatGH()
    solver.append_formula(cnf)
    
    def filter_model(model):
        filter_literals = [variables[lit] for lit in model if lit > 0]
        solved_sudoku = deepcopy(sudoku)
        for (i, j, l) in filter_literals:
            solved_sudoku[i][j] = l
        return solved_sudoku

    def create_generator():
        for model in solver.enum_models():
            yield filter_model(model)

    # solutions = (filter_model(model) for model in solver.enum_models())
    solutions = create_generator()

    sample_solution = next(solutions)
    print("Input:")
    print(pretty_repr(sudoku, k=k))
    print()
    print("Solution:")
    print(pretty_repr(sample_solution, k=k))
    print()

In [47]:
solve_sudoku_SAT(sudoku)

CNF details:
Number of variables: 729
Number of clauses: 7890
Number of literals: 18038
Number of clauses per variable: 0.09239543726235741

Input:
.-------.-------.-------.
|       |       |       | 
|   3   |   8   |       | 
|       |       |       | 
.-------.-------.-------.
|       |   4   |     5 | 
|       |       |   2 7 | 
|       |       | 3     | 
.-------.-------.-------.
|       |       | 9     | 
|       |   5 6 |       | 
|       |       |       | 
.-------.-------.-------.

Solution:
.-------.-------.-------.
| 2 9 5 | 7 6 4 | 1 3 8 | 
| 4 3 7 | 1 8 9 | 5 6 2 | 
| 1 8 6 | 2 3 5 | 7 4 9 | 
.-------.-------.-------.
| 9 6 2 | 3 4 7 | 8 1 5 | 
| 3 4 1 | 5 9 8 | 6 2 7 | 
| 5 7 8 | 6 2 1 | 3 9 4 | 
.-------.-------.-------.
| 6 5 3 | 4 7 2 | 9 8 1 | 
| 8 1 4 | 9 5 6 | 2 7 3 | 
| 7 2 9 | 8 1 3 | 4 5 6 | 
.-------.-------.-------.



In [272]:
# Notation: P_ijl is a propositional variable that is true iff the cell at row i, column j has value l.
vpool = IDPool(start_from=1)
P = lambda i, j, l: vpool.id('P{0}@{1}@{2}'.format(i, j, l))

# initialize the formula
cnf = CNF()

# Constrain 1A: Each cell must contain a value between 1 and N.
for i in range(N):
    for j in range(N):
        cnf.append([P(i, j, l) for l in range(1, N + 1)])

# Constrain 1B; Each cell contains at most 1 number.
for i in range(N):
    for j in range(N):
        for l in range(1, N):
            for m in range(l + 1, N + 1):
                cnf.append([-P(i, j, l), -P(i, j, m)])

# Constraint 2: Each row contains all numbers.
# For each row i, for each number l, there must be a cell in row i that contains l.
for i in range(N):
    for l in range(1, N + 1):
        cnf.append([P(i, j, l) for j in range(N)])

# Constaint 3: Each column contains all numbers.
# For each column j, for each number l, there must be a cell in column j that contains l.
for j in range(N):
    for l in range(1, N + 1):
        cnf.append([P(i, j, l) for i in range(N)])

# Constraint 4: Each kxk sub-grid contains all numbers.
for I in range(k):
    for J in range(k):
        i_range = range(I * k, (I + 1) * k)
        j_range = range(J * k, (J + 1) * k)
        for l in range(1, N + 1):
            cnf.append([P(i, j, l) for i in i_range for j in j_range])


# Constraint 5: For no cell, the same value occurs in its neighborhood.
for i in range(N):
    for j in range(N):
        ij_ranges = [
            [max(i - 1, 0), i, min(n - 1, i + 1)],
            [max(j - 1, 0), j, min(n - 1, j + 1)],
        ]
        ngbd = itertools.product(*somelists)
        for l in range(1, N + 1):
            for (i_, j_) in ngbd:
                if i_ == i and j_ == j:
                    continue
                cnf.append([-P(i, j, l), -P(i_, j_, l)])

# Constraint 6: All zig-zag paths are clear of having duplicates.
# Part A: north-west to south-east
indices = []
for i in range(N):
    i_pivot, j_pivot = i, 0
    for l in range(1, N + 1):
        path_indices = []
        t = 1
        while t < N and (i + t < N):
            # if P(i_pivot, j_pivot, l), then none of the others 
            # can have value l at the same time.
            cnf.append([-P(i_pivot, j_pivot, l), -P(i + 1, t, l)])
            t += 1

for j in range(N):
    i_pivot, j_pivot = 0, j
    for l in range(1, N + 1):
        t = 1
        while t < N and t + j < N:
            cnf.append([-P(i_pivot, j_pivot, l), -P(t, j + t, l)])
            # matrix[t][j + t] = j
            t += 1

# Part B: south-west to north-east
for i in range(N):
    i_pivot, j_pivot = i, 0
    for l in range(1, N + 1):
        t = 1
        while t < N and t <= i:
            cnf.append([-P(i_pivot, j_pivot, l), -P(i - t, t, l)])
            # matrix[i - t][t] = i
            t += 1

for j in range(N):
    i_pivot, j_pivot = 0, j
    for l in range(1, N + 1):
        t = 1
        while t < N and (j + t < N):
            cnf.append([-P(i_pivot, j_pivot, l), -P(N - 1 - t, j + t, l)])
            # matrix[N - 1 - t][j + t] = j
            t += 1

# Contraint 7: Retain values provided in the input sudoku.
for i in range(N):
    for j in range(N):
        if sudoku[i][j] != 0:
            cnf.append([P(i, j, sudoku[i][j])])

# print(cnf.nv), print(len(cnf.clauses))
describe_cnf(cnf)

Number of variables: 729
Number of clauses: 5295
Number of literals: 12819
Number of clauses per variable: 0.1376770538243626


In [284]:
from pysat.solvers import MinisatGH
solver = MinisatGH()
solver.append_formula(cnf)

for i, model in enumerate(solver.enum_models(), 2):
    print("MODEL #{}:".format(i))
    for lit in model:
        if lit > 0:
            print("Variable {} is true".format(lit))
        else:
            print("Variable {} is false".format(-1*lit))

In [286]:
for i, model in enumerate(solver.enum_models(), 2):
    print(i, model)

In [280]:
solver.get_core()

In [228]:

pigeon = lambda i, j: vpool.id('pigeon{0}@{1}'.format(i, j))

In [233]:
pigeon(2, 10)

5

In [241]:
cnf.append([P(2, 2000, 1)])

In [242]:
cnf

In [243]:
cnf.nv

211